This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [18]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'My API Key'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [19]:
import requests
import json
from collections import defaultdict


Note: API's can change a bit with each version, for this exercise it is reccomended to use the "V3" quandl api at `https://www.quandl.com/api/v3/`

In [20]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?rows=1&api_key=' + API_KEY
data = requests.get(url)
data_json = data.json()


In [21]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(type(data_json))
print(data_json)
print(data_json['dataset_data']['data'])
print(data_json['dataset_data']['column_names'])

<class 'dict'>
{'dataset_data': {'limit': 1, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2000-06-07', 'end_date': '2020-12-01', 'frequency': 'daily', 'data': [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]], 'collapse': None, 'order': None}}
[['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]]
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [22]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY
data = requests.get(url)
data_dict = data.json()


In [23]:
# Checking top record in the daily data 
print(data_dict['dataset_data']['data'][0])

# Creating a dictionary with dates as keys and daily price & volume information as values in a list
daily_values = defaultdict(list)
for value in data_dict['dataset_data']['data']:
    daily_values[value[0]] = value[1:]

['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


In [24]:
# Initializing target variables
highest_opening_price = 0.0
lowest_opening_price = 1000000.0
largest_intra_day_change = 0.0
largest_inter_day_change = 0.0
no_of_trading_days = 0.0
traded_volume = []
prev_close = daily_values['2017-12-29'][3]
todays_change = 0.0

# Passing None type to a variable 'Nonetype' for checking & avoiding values that are None type
Nonetype = type(None)


def median(list_of_values):
    """Return the median of a list of nos."""
    no_of_values = len(list_of_values)
    sorted_list = sorted(list_of_values,key=float)
    if no_of_values != 0 and no_of_values % 2 != 0:
        return sorted_list[int((no_of_values-1)/2)]
    else:
        return (sorted_list[int((no_of_values)/2)]+sorted_list[int((no_of_values-2)/2)])/2


# Parsing over dict to calculate target values
for i,v in daily_values.items():
    no_of_trading_days += 1
    if type(v[0]) != Nonetype:
        if v[0] > highest_opening_price:  # Calculates highest opening price 
            highest_opening_price = v[0]
        if v[0] < lowest_opening_price: # Calculates lowest opening price
            lowest_opening_price =v[0]
    
    if type(v[1]) != Nonetype and type(v[2]) != Nonetype: # Calculates largest change in any one day
        change = v[1] - v[2]
        if change > largest_intra_day_change:
            largest_intra_day_change = change
            
    if type(v[6]) != Nonetype:       # Adds traded volumes into a list for calculating mean & median later
        traded_volume.append(v[6])
    
    if type(v[3]) != Nonetype:       # Calculates largest change between any two days
        todays_change = abs(v[3] - prev_close)
        if todays_change > largest_inter_day_change:
            largest_inter_day_change = todays_change
        prev_close = v[3]

average_traded_volume = sum(traded_volume)/len(traded_volume)
median_traded_volume = median(traded_volume)

print(f"Highest Opening Price : {highest_opening_price:5.2f}")
print(f"Lowest Opening Price : {lowest_opening_price:1.2f}")
print(f"Largest change within any one day : {largest_intra_day_change:1.2f}")
print(f"Largest change between any two successive days : {largest_inter_day_change:1.2f}")
print(f"Average Traded Volume in 2017 : {average_traded_volume:1.0f}")
print(f"Median Traded Volume in 2017 : {median_traded_volume:1.0f}")

Highest Opening Price : 53.11
Lowest Opening Price : 34.00
Largest change within any one day : 2.81
Largest change between any two successive days : 2.56
Average Traded Volume in 2017 : 3853589
Median Traded Volume in 2017 : 3292223
